In [ ]:
!python --version

In [ ]:
!pip install concrete-ml

In [ ]:
!pip install pandas

In [ ]:
!pip install --upgrade jupyter

In [ ]:
!pip install --upgrade ipywidgets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from concrete.ml.sklearn import LogisticRegression as ConcreteLogisticRegression

# Load dataset
data = datasets.load_breast_cancer()
bc = pd.DataFrame(data.data, columns=data.feature_names)
bc['class'] = data.target

In [ ]:
# Select a smaller subset of features
selected_features_5 = [
    'mean compactness',
    'mean concave points',
    'radius error',
    'area error',
    'worst texture'
]

In [ ]:
# Prepare the data with selected features
bc_reduced = bc[selected_features_5]
X, y = bc_reduced.to_numpy(), data.target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Preprocess the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Train Sklearn Logistic Regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict and evaluate
y_pred_test = np.asarray(logreg.predict(X_test))
sklearn_acc = np.sum(y_pred_test == y_test) / len(y_test) * 100
print(f"Regular Sklearn model accuracy: {sklearn_acc:.4f}%")

In [ ]:
n_bits_dict = {
    "net_inputs": 5,
    "op_weights": 2,
    "net_outputs": 5,
    "op_inputs": 5
}

# Train Concrete Logistic Regression model
q_logreg = ConcreteLogisticRegression(n_bits=n_bits_dict)
q_logreg.fit(X_train, y_train)

In [ ]:
# Compile the model
try:
    q_logreg.compile(X_train)
except RuntimeError as e:
    print("Compilation error:", e)

In [ ]:
# Evaluate quantized model
q_y_pred_test = q_logreg.predict(X_test)
quantized_accuracy = (q_y_pred_test == y_test).mean() * 100
print(f"Clear quantized model accuracy: {quantized_accuracy:.4f}%")

# Evaluate homomorphic model
q_y_pred_fhe = q_logreg.predict(X_test, execute_in_fhe=True)
homomorphic_accuracy = (q_y_pred_fhe == y_test).mean() * 100
print(f"Homomorphic model accuracy: {homomorphic_accuracy:.4f}%")